In [1]:
!nvidia-smi

Sat Dec 17 07:35:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    52W / 400W |      0MiB / 40960MiB |     27%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
class my_args:
    def __init__(self, generator_name, dataset_name, dataset_percent, num_cands, scorer_path):
        self.generator_name = generator_name
        self.dataset_name = dataset_name
        self.num_cands = dataset_percent
        self.dataset_percent = num_cands
        self.scorer_path = scorer_path
args = my_args('tuned_t5_model','clean_covid.csv',16,10, 'cache/22-12-16-0/scorer.bin') # 10 stands for 10% of dataset are used

In [3]:
!python gen_candidate.py --generator_name {args.generator_name} --dataset_name {args.dataset_name} --dataset_percent {args.dataset_percent} --num_cands {args.num_cands} --cand_gen_batch 64

start tokenize dataset--------------------
100%|█████████████████████████████████████████████| 9/9 [00:03<00:00,  2.69ba/s]
start generate training candidates!--------------------------------------------------
start generate validation candidates!--------------------------------------------------
start generate testing candidates!--------------------------------------------------
start saving candidate summariesch_index: 128, last epoch time: 14 sec


In [5]:
# create folder for processed dataset
os.makedirs(f'candidates/{args.generator_name}_{args.num_cands}_processed/diverse/train', exist_ok=True)
os.makedirs(f'candidates/{args.generator_name}_{args.num_cands}_processed/diverse/test', exist_ok=True)
os.makedirs(f'candidates/{args.generator_name}_{args.num_cands}_processed/diverse/val', exist_ok=True)

# process candidates
print('start processing val candidates for training')
!python preprocess.py --src_dir candidates/{args.generator_name}_{args.num_cands}/diverse --tgt_dir candidates/{args.generator_name}_{args.num_cands}_processed/diverse/val --split 'val' --cand_num {args.num_cands}
print('start processing train candidates for training')
!python preprocess.py --src_dir candidates/{args.generator_name}_{args.num_cands}/diverse --tgt_dir candidates/{args.generator_name}_{args.num_cands}_processed/diverse/train --split 'train' --cand_num {args.num_cands}
print('start processing test candidates for training')
!python preprocess.py --src_dir candidates/{args.generator_name}_{args.num_cands}/diverse --tgt_dir candidates/{args.generator_name}_{args.num_cands}_processed/diverse/test --split 'test' --cand_num {args.num_cands}

start processing val candidates for training
finish
start processing train candidates for training
finish
start processing test candidates for training
finish


In [6]:
#make necessary dirs for training
if not os.path.exists('cache'):
    os.makedirs('cache')

training_data_path = f'candidates/{args.generator_name}_{args.num_cands}_processed'

In [7]:
!python main.py --cuda --gpuid 0 -l --dataset {training_data_path} --model_type 'xlm-roberta-base'

Downloading: 100%|██████████████████████████████| 615/615 [00:00<00:00, 643kB/s]
Downloading: 100%|█████████████████████████| 5.07M/5.07M [00:00<00:00, 20.4MB/s]
Downloading: 100%|█████████████████████████| 9.10M/9.10M [00:00<00:00, 30.5MB/s]
Downloading: 100%|█████████████████████████| 1.12G/1.12G [00:14<00:00, 76.2MB/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [8]:
!python main.py --cuda --gpuid 0 -l --dataset {training_data_path} --model_type 'microsoft/deberta-base'

Downloading: 100%|███████████████████████████| 52.0/52.0 [00:00<00:00, 52.8kB/s]
Downloading: 100%|██████████████████████████████| 474/474 [00:00<00:00, 453kB/s]
Downloading: 100%|███████████████████████████| 899k/899k [00:00<00:00, 5.71MB/s]
Downloading: 100%|███████████████████████████| 456k/456k [00:00<00:00, 3.96MB/s]
Downloading: 100%|███████████████████████████| 559M/559M [00:07<00:00, 73.2MB/s]
Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel fro

In [ ]:
!python main.py --cuda --gpuid 0 -l --dataset {training_data_path} --model_type 'google/electra-small-discriminator'

In [ ]:
!python main.py --cuda --gpuid 0 -l --dataset {training_data_path} --model_type 'bert-base-uncased'

In [ ]:
# start training scorer
training_data_path = f'candidates/{args.generator_name}_{args.num_cands}_processed'
!python main.py --cuda --gpuid 0 -l --dataset {training_data_path}
# default training for 5 epoch

In [9]:
# start evaluating scorer just trained
!python evaluate_model.py --generator_name {args.generator_name} --dataset_name {args.dataset_name} --scorer_path 'cache/22-12-17-0-xlm-roberta-base/scorer.bin' --dataset_percent 1 --scorer_architecture_name 'xlm-roberta-base'

start tokenize data
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.82ba/s]
There are 814 samples in test set!
start loading scorer model
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
start evaluating, total_num of samples is:814
Truncation was not explicitly activated but `max_length` is provided a spec